In [2]:
import geopandas as gpd
import cartopy.feature as cfeature
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.transform import array_bounds
import ultraplot as uplt
import cartopy.crs as ccrs
import rasterio as rio
import numpy as np
import matplotlib.patches as mpatches

In [3]:
new_york_2021 = rio.open('./Data/Yearly Images/upscaled_class_2021.tif', 'r')
aerial = rio.open('./Data/Yearly Images/upscaled_aerial_survey_2021.tif', 'r')

RasterioIOError: ./Data/Yearly Images/upscaled_class_2021.tif: No such file or directory

In [ ]:
dest_crs = ccrs.LambertConformal(central_longitude=-76.0, central_latitude=42.0,)

def prepare_for_viz(raster_file, dest_crs):
    transform, width, height = calculate_default_transform(
        raster_file.crs, dest_crs, raster_file.width, raster_file.height, *raster_file.bounds)
    
    new_image, new_transform = reproject(
        source=raster_file.read(),
        src_transform=raster_file.transform,
        src_crs=raster_file.crs,
        dst_crs=dest_crs,
        resampling=Resampling.nearest,
        dst_nodata=np.nan)
    print(new_transform)
    new_bounds = array_bounds(new_image.shape[1], new_image.shape[2], new_transform)
    
    return new_image, new_bounds, new_transform

# 2021
image_2021, bounds_2021, transform_2021 = prepare_for_viz(new_york_2021, dest_crs)

In [ ]:
def prepare_for_viz2(raster_file, dest_crs, dest_transform):
    transform, width, height = calculate_default_transform(
        raster_file.crs, dest_crs, raster_file.width, raster_file.height, *raster_file.bounds)
    
    new_image, new_transform = reproject(
        source=raster_file.read(),
        src_transform=raster_file.transform,
        src_crs=raster_file.crs,
        dst_transform=dest_transform,
        dst_crs=dest_crs,
        resampling=Resampling.nearest,
        dst_nodata=0)
    print(new_transform)
    new_bounds = array_bounds(new_image.shape[1], new_image.shape[2], new_transform)
    
    return new_image, new_bounds

# Aerial Survey
image_aerial, bounds_aerial = prepare_for_viz2(aerial, dest_crs, transform_2021)

# Properly mask out pixels
image_aerial = image_aerial.astype(float)
mask = image_aerial == 0
image_aerial[mask] = np.nan
image_aerial -= 1

In [ ]:
def add_raster_data(ax, image, bounds, year, crs, vmin, vmax, cmap):
    m = ax.imshow(image[0], cmap=cmap, levels=np.linspace(vmin, vmax, 11), transform=crs,
                  extent=[bounds[0], bounds[2], bounds[1], bounds[3]])
    ax.format(grid=False, facecolor='white', title=year)
    ax.set_extent([-80, -71.7, 40.3, 45], crs=ccrs.PlateCarree())
    
    return m

In [ ]:
cmap = ['gray2', 'blood red']

In [ ]:
fig, axes = pplt.subplots(nrows=1, ncols=2, figsize=('140mm','70mm'), proj=dest_crs, facecolor='white', fontsize=12)

axes.format(abc='A.', abcloc='ul')

vmax = 1
vmin = 0

# Yearly raster images
add_raster_data(axes[0], threshold_2021, bounds_2021, 'Satellite - 2021', dest_crs, vmin, vmax, cmap)
# Aerial survey data
add_raster_data(axes[1], image_aerial, bounds_aerial, 'Aerial Survey - 2021', dest_crs, vmin, vmax, cmap)

# Add counties and set extent
#for i in range(6):
    #axes[i].add_geometries(county_2021.geometry, crs=county_2021.crs, facecolor='None', edgecolor='black', alpha=0.8)
    
no_patch = mpatches.Patch(color=cmap[0], label='Unaffected')
defol_patch = mpatches.Patch(color=cmap[1], label='Defoliation')
fig.legend(handles=[no_patch, defol_patch], loc='b', ncols=2)

#fig.savefig('./Figures/Yearly Images/abstract_figure_erdgvm.png')

In [ ]:
image_aerial.shape

In [ ]:
threshold_2021.shape

In [ ]:
bounds_2021

In [ ]:
bounds_aerial

In [ ]:
from contextlib import contextmanager  
import rasterio as rio
from rasterio.io import MemoryFile
from rasterio.windows import from_bounds

# Rasterio is a bit weird and doesn't allow reprojections that explicitly change the transform unless they are
# to a destination file. Therefore, we create an in memory file to complete the reprojection and yield
# the new file for subsequent operations. The conextmanager decorator allows this to be used in a `with` statement.
@contextmanager  
def reproject_raster(in_path, crs):
    with rio.open(in_path, 'r') as src:
        # Calculate transform in new crs if unspecified
        transform, width, height = calculate_default_transform(src.crs, crs, src.width, src.height, *src.bounds)
        
        # Create properities for MemoryFile
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': crs,
            'transform': transform,
            'width': width,
            'height': height})

        with MemoryFile() as memfile:
            with memfile.open(**kwargs) as dst:
                for i in range(1, src.count + 1):
                    reproject(
                        source=rio.band(src, i),
                        destination=rio.band(dst, i),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=crs,
                        resampling=Resampling.nearest,
                        dst_nodata=np.nan)
            with memfile.open() as dataset:  # Reopen as DatasetReader
                yield dataset  # Note yield not return as we're a contextmanager

In [ ]:
with reproject_raster('./Data/Yearly Images/upscaled_class_2021.tif', dest_crs) as in_mem_ds:
    transform = in_mem_ds.transform
    width = in_mem_ds.width
    height = in_mem_ds.height
    new_york_raster = in_mem_ds.read()
    new_york_bounds = in_mem_ds.bounds
    
with reproject_raster('./Data/Yearly Images/upscaled_aerial_survey_2021.tif', dest_crs) as in_mem_ds:
    small_window = from_bounds(*new_york_bounds, transform=in_mem_ds.transform)
    aerial_raster = in_mem_ds.read(out_shape=(1, height, width), window=small_window, boundless=True)
    
    # Properly mask out pixels
    aerial_raster = aerial_raster.astype(float)
    mask = aerial_raster == 0
    aerial_raster[mask] = np.nan
    aerial_raster -= 1

In [ ]:
cutoff = 0.3

threshold_2021 = new_york_raster.copy()
threshold_2021 = np.greater(new_york_raster, cutoff, out=threshold_2021, where=~np.isnan(new_york_raster))

In [ ]:
fig, axes = pplt.subplots(nrows=1, ncols=2, figsize=('140mm','70mm'), proj=dest_crs, facecolor='white', fontsize=12)

axes.format(abc='A.', abcloc='ul')

vmax = 1
vmin = 0

# Aerial survey data
add_raster_data(axes[1], aerial_raster, new_york_bounds, 'Aerial Survey - 2021', dest_crs, vmin, vmax, cmap)

# Yearly raster images
add_raster_data(axes[0], threshold_2021, new_york_bounds, 'Satellite - 2021', dest_crs, vmin, vmax, cmap)


# Add counties and set extent
#for i in range(6):
    #axes[i].add_geometries(county_2021.geometry, crs=county_2021.crs, facecolor='None', edgecolor='black', alpha=0.8)
    
no_patch = mpatches.Patch(color=cmap[0], label='Unaffected')
defol_patch = mpatches.Patch(color=cmap[1], label='Defoliation')
fig.legend(handles=[no_patch, defol_patch], loc='b', ncols=2)

#fig.savefig('./Figures/Yearly Images/abstract_figure_erdgvm.png')

In [ ]:
threshold_2021.shape

In [ ]:
np.unique(threshold_2021)

In [ ]:
test = (np.where((threshold_2021 == 0) & (aerial_raster == 0), 1, 0) +
        np.where((threshold_2021 == 1) & (aerial_raster == 1), 2, 0) +
        np.where((threshold_2021 == 1) & (aerial_raster == 0), 3, 0) +
        np.where((threshold_2021 == 0) & (aerial_raster == 1), 4, 0))

In [ ]:
fig, axes = pplt.subplots(nrows=1, ncols=1, figsize=('140mm','70mm'), proj=dest_crs, facecolor='white', fontsize=12)

axes.format(abc='A.', abcloc='ul')

vmax = 4
vmin = 0

# Aerial survey data
add_raster_data(axes[0], test, new_york_bounds, 'Aerial Survey - 2021', dest_crs, vmin, vmax, ['white', 'gray2', 'green', 'blue', 'pink'])


# Add counties and set extent
#for i in range(6):
    #axes[i].add_geometries(county_2021.geometry, crs=county_2021.crs, facecolor='None', edgecolor='black', alpha=0.8)
    
no_patch = mpatches.Patch(color=cmap[0], label='Unaffected')
defol_patch = mpatches.Patch(color=cmap[1], label='Defoliation')
fig.legend(handles=[no_patch, defol_patch], loc='b', ncols=2)

#fig.savefig('./Figures/Yearly Images/abstract_figure_erdgvm.png')

In [ ]:
counts = np.histogram(test, bins=[-0.5, 0.5, 1.5, 2.5, 3.5, 4.5])[0]
print(counts[2] / (counts[2]+counts[4]))
print(counts[2] / (counts[2]+counts[3]))

In [ ]:
transform

In [ ]:
transform[2]

In [ ]:
coarse_transform = rio.Affine(1000, 0.0, transform[2], 0.0, 1000, transform[5])

In [ ]:
with reproject_raster('./Data/Yearly Images/upscaled_class_2021.tif', dest_crs) as in_mem_ds:
    small_window = from_bounds(*new_york_bounds, transform=in_mem_ds.transform)
    
    coarse_height = height // 4
    coarse_width = width // 4
    
    new_york_raster = in_mem_ds.read(out_shape=(1, coarse_height, coarse_width), window=small_window, boundless=True)
    
with reproject_raster('./Data/Yearly Images/upscaled_aerial_survey_2021.tif', dest_crs) as in_mem_ds:
    small_window = from_bounds(*new_york_bounds, transform=in_mem_ds.transform)
    aerial_raster = in_mem_ds.read(out_shape=(1, coarse_height, coarse_width), window=small_window, boundless=True)
    
    # Properly mask out pixels
    aerial_raster = aerial_raster.astype(float)
    mask = aerial_raster == 0
    aerial_raster[mask] = np.nan
    aerial_raster -= 1

In [ ]:
new_york_raster.shape

In [ ]:
cutoff = 0.2

threshold_2021 = new_york_raster.copy()
threshold_2021 = np.greater(new_york_raster, cutoff, out=threshold_2021, where=~np.isnan(new_york_raster))

In [ ]:
fig, axes = pplt.subplots(nrows=1, ncols=2, figsize=('140mm','70mm'), proj=dest_crs, facecolor='white', fontsize=12)

axes.format(abc='A.', abcloc='ul')

vmax = 1
vmin = 0

# Aerial survey data
add_raster_data(axes[1], aerial_raster, new_york_bounds, 'Aerial Survey - 2021', dest_crs, vmin, vmax, cmap)

# Yearly raster images
add_raster_data(axes[0], threshold_2021, new_york_bounds, 'Satellite - 2021', dest_crs, vmin, vmax, cmap)


# Add counties and set extent
#for i in range(6):
    #axes[i].add_geometries(county_2021.geometry, crs=county_2021.crs, facecolor='None', edgecolor='black', alpha=0.8)
    
no_patch = mpatches.Patch(color=cmap[0], label='Unaffected')
defol_patch = mpatches.Patch(color=cmap[1], label='Defoliation')
fig.legend(handles=[no_patch, defol_patch], loc='b', ncols=2)

#fig.savefig('./Figures/Yearly Images/abstract_figure_erdgvm.png')

In [ ]:
test = (np.where((threshold_2021 == 0) & (aerial_raster == 0), 1, 0) +
        np.where((threshold_2021 == 1) & (aerial_raster == 1), 2, 0) +
        np.where((threshold_2021 == 1) & (aerial_raster == 0), 3, 0) +
        np.where((threshold_2021 == 0) & (aerial_raster == 1), 4, 0))

In [ ]:
counts = np.histogram(test, bins=[-0.5, 0.5, 1.5, 2.5, 3.5, 4.5])[0]
print(counts[2] / (counts[2]+counts[4]))
print(counts[2] / (counts[2]+counts[3]))

In [ ]:
with reproject_raster("./Data/site_validation/as_satellite_comp_2021_10000.tif", dest_crs) as in_mem_ds:
    comp = in_mem_ds.read()
    comp_bounds = in_mem_ds.bounds

In [ ]:
comp.shape

In [ ]:
np.unique(comp[0])

In [ ]:
comp[1]

In [ ]:
cutoff = 0.05

relative_comp = comp[[0]] / (comp[[1]]+0.00001)
threshold_2021 = relative_comp.copy()
threshold_2021 = np.greater(relative_comp, cutoff, out=threshold_2021, where=~np.isnan(relative_comp))

In [ ]:
fig, axes = pplt.subplots(nrows=1, ncols=2, figsize=('140mm','70mm'), proj=dest_crs, facecolor='white', fontsize=12)

axes.format(abc='A.', abcloc='ul')

vmax = 1
vmin = 0

# Aerial survey data
add_raster_data(axes[1], comp[[2]], comp_bounds, 'Aerial Survey - 2021', dest_crs, vmin, vmax, cmap)

# Yearly raster images
add_raster_data(axes[0], threshold_2021, comp_bounds, 'Satellite - 2021', dest_crs, vmin, vmax, cmap)


# Add counties and set extent
#for i in range(6):
    #axes[i].add_geometries(county_2021.geometry, crs=county_2021.crs, facecolor='None', edgecolor='black', alpha=0.8)
    
no_patch = mpatches.Patch(color=cmap[0], label='Unaffected')
defol_patch = mpatches.Patch(color=cmap[1], label='Defoliation')
fig.legend(handles=[no_patch, defol_patch], loc='b', ncols=2)

#fig.savefig('./Figures/Yearly Images/abstract_figure_erdgvm.png')

In [ ]:
# Test potential cutoffs
cutoff = 0.2

for cutoff in [0.01, 0.02, 0.03, 0.04, 0.05]:
    relative_comp = comp[[0]] / (comp[[1]]+0.00001)
    threshold_2021 = relative_comp.copy()
    threshold_2021 = np.greater(relative_comp, cutoff, out=threshold_2021, where=~np.isnan(relative_comp))
    
    test = (np.where((threshold_2021 == 0) & (comp[[2]] == 0), 1, 0) +
            np.where((threshold_2021 == 1) & (comp[[2]] == 2021), 2, 0) +
            np.where((threshold_2021 == 1) & (comp[[2]] == 0), 3, 0) +
            np.where((threshold_2021 == 0) & (comp[[2]] == 2021), 4, 0))
    
    counts = np.histogram(test, bins=[-0.5, 0.5, 1.5, 2.5, 3.5, 4.5])[0]
    print(f"Cutoff: {cutoff}, PA: {counts[2] / (counts[2]+counts[4]):.3f}, UA: {counts[2] / (counts[2]+counts[3]):.3f}")